In [1]:
import os
import sys
import pandas as pd
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkContext, SparkConf

config = SparkConf()
config.set("spark.app.name", "GF_spark_app")

sc = SparkContext(conf=config)

In [3]:
sc

<SparkContext master=yarn appName=GF_spark_app>

In [4]:
item = sc.textFile('/labs/laba01/ml-100k/u.item').map(lambda x: x.split("|"))

In [5]:
data = sc.textFile('/labs/laba01/ml-100k/u.data').map(lambda x: x.split("\t"))

In [6]:
df_item = pd.DataFrame(item.collect(), columns=[
    'movie id',
    'movie title',
    'release date',
    'video release date',
    'IMDb URL',
    'unknown',
    'Action',
    'Adventure',
    'Animation',
    "Children's",
    'Comedy',
    'Crime',
    'Documentary',
    'Drama',
    'Fantasy',
    'Film-Noir',
    'Horror',
    'Musical',
    'Mystery',
    'Romance',
    'Sci-Fi',
    'Thriller',
    'War',
    'Western'
])

In [7]:
df_data = pd.DataFrame(data.collect(), columns = [
    'user id',
    'item id',
    'rating',
    'timestamp'
])

In [8]:
df_item[df_item['movie id'].astype('int') == 328]

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
327,328,Conspiracy Theory (1997),08-Aug-1997,,http://us.imdb.com/M/title-exact?Conspiracy+Th...,0,1,0,0,0,...,0,0,0,0,1,1,0,1,0,0


In [9]:
df = df_data[df_data['item id'].astype('int') == 328]

In [10]:
df = df.groupby('rating').count()['item id']

In [11]:
df.name = 'hist_film'

In [12]:
df2 = df_data.groupby('rating').count()['item id']

In [13]:
df2.name = 'hist_all'

In [14]:
result = pd.concat([df,df2], axis=1)

In [15]:
result.index.name = None

In [20]:
res = {}
for i in result:
    l = []
    for j in result[i]:
        l.append(j)
    res[i] = l
! cat lab01.json

{'hist_film': [12, 40, 94, 109, 40], 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [17]:
! touch lab01.json
file = open("lab01.json", "w")
file.write(str(res))
file.close()

In [18]:
sc.stop()

In [19]:
res

{'hist_film': [12, 40, 94, 109, 40],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}